In [ ]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

## Debugging requirements
import pdb

## Performance metrics requirements
import time

## Kernel SVM requirements
from cvxopt import matrix
from cvxopt import solvers
from scipy.spatial.distance import cdist
from numpy.core.defchararray import not_equal

## Exploring possibilities through sklearn
from sklearn import svm

## 1.Loading the data + sanity checks

In [ ]:
def load_data(dsID, set_type='tr', folder_name='data'):
    Xdata_file = folder_name + '/X' + set_type + str(dsID) + '.csv'
    X = pd.read_csv(Xdata_file, header=None, names=['Sequence'], dtype={'Sequence': np.unicode_})
    if set_type=='tr':
        Ydata_file = folder_name + '/Y' + set_type + str(dsID) + '.csv'
        Y = pd.read_csv(Ydata_file, index_col=0, dtype={'Bound': np.dtype(bool)})
        Y.index = Y.index - 1000*dsID
        df = pd.concat([X, Y], axis=1)
    else:
        df = X
    return df

## Loading training data
tr0 = load_data(0, 'tr')
tr1 = load_data(1, 'tr')
tr2 = load_data(2, 'tr')

## Loading test data
te0 = load_data(0, 'te')
te1 = load_data(1, 'te')
te2 = load_data(2, 'te')

## 2. Defining first kernels + running simple classification model

### First kernels

Ok, so here all sequences have the same length. That means that we can start by something simple like Hamming. However, we may want to use something that would seamlessly extend to DNA sequences of different lengths...
Here I will test both the Hamming and the Levenshtein distance as kernels for mapping DNA sequences:

In [ ]:
## Defining both string distances for first kernel tryouts:

def hamming_distance(source, target):
    """Return the Hamming distance between equal-length sequences"""
    if len(source) != len(target):
        raise ValueError("Undefined for sequences of unequal length")
    return np.count_nonzero(not_equal(source,target))

def levenshtein_distance(source, target):
    if len(source) < len(target):
        return levenshtein(target, source)

    # So now we have len(source) >= len(target).
    if len(target) == 0:
        return len(source)

    # We call tuple() to force strings to be used as sequences
    # ('c', 'a', 't', 's') - numpy uses them as values by default.
    source = np.array(tuple(source))
    target = np.array(tuple(target))

    # We use a dynamic programming algorithm, but with the
    # added optimization that we only need the last two rows
    # of the matrix.
    previous_row = np.arange(target.size + 1)
    for s in source:
        # Insertion (target grows longer than source):
        current_row = previous_row + 1

        # Substitution or matching:
        # Target and source items are aligned, and either
        # are different (cost of 1), or are the same (cost of 0).
        current_row[1:] = np.minimum(
                current_row[1:],
                np.add(previous_row[:-1], target != s))

        # Deletion (target grows shorter than source):
        current_row[1:] = np.minimum(
                current_row[1:],
                current_row[0:-1] + 1)

        previous_row = current_row

    return previous_row[-1]


def build_kernel(arr1, arr2, kernel_fct):
    """Builds the kernel matrix from numpy array @arr and kernel function @kernel_fct. V1, unnefficient"""
    try:
        assert len(arr1) > 0
        assert len(arr2) > 0
    except AssertionError:
        print('At least one of the argument arrays is empty')
    if arr1.ndim == 1:
        arr1 = arr1.reshape((len(arr1),1))
    if arr2.ndim == 1:
        arr2 = arr2.reshape((len(arr2),1))
    K = cdist(arr1, arr2, lambda u, v: kernel_fct(list(u[0]),list(v[0])))
    return K

Let's see if we can get anything out of these kernels with SVM by using sklearn:

In [ ]:
clf = svm.SVC(kernel=hamming_distance)
clf.fit(tr0['Sequence'], tr0['Bound'])